In [6]:
import urllib.request, json 
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [7]:
subs = ['depression', 'relationship_advice', 'offmychest',
'IAmA', 'needadvice', 'tifu', 'confessions']

In [ ]:
for sub in subs:
    print('*********')
    print(sub)
    sub_url = 'https://api.pushshift.io/reddit/search/submission/?subreddit=%s'%sub
    with urllib.request.urlopen(sub_url+"&size=1000&sort=desc&before=30d") as url:
        data = json.loads(url.read().decode())
    data = data['data']
    df_sub = pd.DataFrame.from_dict(json_normalize(data), orient='columns')
    created_utc_now_sub = df_sub.tail(1)['created_utc']
    while df_sub.shape[0] < 25000:
        final_url = sub_url+"&size=1000&sort=desc&before=%d"%df_sub.tail(1)['created_utc']  
        with urllib.request.urlopen(final_url) as url:
            data = json.loads(url.read().decode())
        data = data['data']
        df_new_sub = pd.DataFrame.from_dict(json_normalize(data), orient='columns')
        df_sub = df_sub.append(df_new_sub) 
        created_utc_now_sub = df_sub.tail(1)['created_utc']
        if df_sub.shape[0]%5000 == 0:
            print(created_utc_now_sub)
            print(df_sub.shape)
    
    df_sub.to_csv('expanded_%s_preprocessed.csv' %sub)
#    with open('df_sub_%s.pickle'%sub, 'wb') as handle:
#       pickle.dump(df_sub, handle) 